# Notebook responsável por alguns aspectos de modelagem OneToOne / OneToFew Linking (usando MongoEngine)

## Bibliotecas importantes

In [1]:
from mongoengine import *
import pprint
import numpy as np
import pandas as pd
import unidecode

# para geração de dados fake
from faker import Faker


## Classes úteis

In [2]:
class Endereco (Document):
    logradouro = StringField(max_length=200, required=True)
    numero = StringField(max_length=10, required=True)
    bairro = StringField(max_length=50, required=True)
    cidade = StringField(max_length=200, required=True)
    uf = StringField(max_length=50, required=True)
    cep = StringField(max_length=10, required=True)
    
    def __str__(self):
        return f"{self.pk}, {self.logradouro}, {self.numero}"
    
    
class Pessoa (Document):
    cpf = StringField(max_length=20, required=True)
    primeiro_nome = StringField(max_length=200, required=True)
    ultimo_nome = StringField(max_length=400, required=True)
    idade = IntField(min_value=0, max_value=120, required=True)
    data_nascimento = StringField(max_length=20, required=True)
    email = EmailField(max_length=50)
    telefones = ListField(StringField(max_length=50), max_length=4)
    formacoes = ListField(StringField(max_length=50), max_length=4)
    
    # Referência (linking) para um objeto da classe pessoa
    # ref.:https://mongoengine-odm.readthedocs.io/apireference.html#documents
    #endereco = ReferenceField(Endereco, reverse_delete_rule=DENY)
    enderecos = ListField(ReferenceField(Endereco, reverse_delete_rule=DENY))
    
    def __str__(self):
        return f"{self.pk}, {self.cpf}, {self.primeiro_nome}, Endereço: {self.enderecos}"


## Funções uteis

In [3]:
def criar_endereco():
    
    logradouro = fake.street_name()
    numero = fake.building_number()
    bairro = fake.bairro()
    cidade = fake.city()
    uf = fake.administrative_unit()
    cep = fake.postcode()

    endereco = Endereco(logradouro=logradouro, numero=numero, bairro=bairro, cidade=cidade, uf=uf, cep=cep)
    return endereco

In [4]:
def criar_pessoa():
    
    cpf = fake.cpf()
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    idade = np.random.randint(18, 50)
    data_nascimento = fake.date()
    email = f'{cpf}@{fake.free_email_domain()}'
    
    # criando os telefones
    telefones = []
    n = np.random.randint(1, 4)
    for i in range(n):
        telefones.append(fake.msisdn())
    
    # criando as formações
    formacoes = []
    n = np.random.randint(1, 4)
    for i in range(n):
        formacoes.append(fake.job())
    
    enderecos = []
    n = np.random.randint(1, 4)
    for i in range(n):
        enderecos.append(criar_endereco())

    
    pessoa = Pessoa(cpf=cpf, primeiro_nome=primeiro_nome, 
                    ultimo_nome=ultimo_nome, idade=idade, 
                    data_nascimento=data_nascimento, 
                    email=email, telefones=telefones, 
                    formacoes=formacoes, enderecos=enderecos)
    return pessoa

## Configurações

In [5]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de vários documentos

### Conectando ao MongoDB

In [6]:
connect('onetomany_linking')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### Exemplo - Criação de vários documentos compostos

In [7]:
lista_pessoas = []

for i in range(10):
    lista_pessoas.append(criar_pessoa())
    
pp.pprint(lista_pessoas)

[<Pessoa: None, 618.259.073-40, Lara, Endereço: [<Endereco: None, Sítio Emanuelly Gonçalves, 6>, <Endereco: None, Recanto de Teixeira, 729>]>,
 <Pessoa: None, 271.864.039-13, Isabella, Endereço: [<Endereco: None, Chácara de Costela, 93>, <Endereco: None, Fazenda Fernando Rocha, 551>, <Endereco: None, Recanto de da Rocha, 28>]>,
 <Pessoa: None, 792.158.634-19, Bruna, Endereço: [<Endereco: None, Jardim Moura, 2>]>,
 <Pessoa: None, 912.750.436-07, Kamilly, Endereço: [<Endereco: None, Praia de Jesus, 714>]>,
 <Pessoa: None, 740.352.619-80, Brenda, Endereço: [<Endereco: None, Conjunto Stephany Ferreira, 8>, <Endereco: None, Avenida de Cardoso, 54>]>,
 <Pessoa: None, 784.953.602-92, Maysa, Endereço: [<Endereco: None, Fazenda de Correia, 55>, <Endereco: None, Condomínio Bruno Pinto, 712>, <Endereco: None, Via Fernanda Rodrigues, 39>]>,
 <Pessoa: None, 970.485.316-57, Milena, Endereço: [<Endereco: None, Área Renan Nunes, 74>]>,
 <Pessoa: None, 780.921.563-95, Luiz Felipe, Endereço: [<Endereco:

In [8]:
pp.pprint(lista_pessoas[0].pk)

None


### Inserindo os documentos no banco de dados

In [9]:
# inserindo 1 documento apenas
for e in lista_pessoas[0].enderecos:
    e.save()
lista_pessoas[0].save()

<Pessoa: 625ff9298ed61a282c640b7f, 618.259.073-40, Lara, Endereço: [<Endereco: 625ff9298ed61a282c640b7d, Sítio Emanuelly Gonçalves, 6>, <Endereco: 625ff9298ed61a282c640b7e, Recanto de Teixeira, 729>]>

In [10]:
pp.pprint(lista_pessoas[0].pk)

ObjectId('625ff9298ed61a282c640b7f')


In [11]:
pp.pprint(lista_pessoas[0].enderecos)

[<Endereco: 625ff9298ed61a282c640b7d, Sítio Emanuelly Gonçalves, 6>,
 <Endereco: 625ff9298ed61a282c640b7e, Recanto de Teixeira, 729>]


In [12]:
# inserindo todos os outros documentos
for p in lista_pessoas[1:]:
    for e in p.enderecos:
        e.save()
    p.save()

## Recuperando os documentos inseridos

#### Recuperar todos

In [13]:
for p in Pessoa.objects:
    pp.pprint(p.id)

ObjectId('625ff9298ed61a282c640b7f')
ObjectId('625ff9298ed61a282c640b83')
ObjectId('625ff9298ed61a282c640b85')
ObjectId('625ff9298ed61a282c640b87')
ObjectId('625ff9298ed61a282c640b8a')
ObjectId('625ff9298ed61a282c640b8e')
ObjectId('625ff9298ed61a282c640b90')
ObjectId('625ff9298ed61a282c640b92')
ObjectId('625ff9298ed61a282c640b95')
ObjectId('625ff9298ed61a282c640b98')


In [14]:
pessoa = Pessoa.objects.first()
pp.pprint(pessoa)

<Pessoa: 625ff9298ed61a282c640b7f, 618.259.073-40, Lara, Endereço: [<Endereco: 625ff9298ed61a282c640b7d, Sítio Emanuelly Gonçalves, 6>, <Endereco: 625ff9298ed61a282c640b7e, Recanto de Teixeira, 729>]>


In [15]:
pp.pprint(pessoa.enderecos[0].pk)

ObjectId('625ff9298ed61a282c640b7d')


In [16]:
endereco = Endereco.objects(id='625f5015cc40c07d0d148253').first()
endereco.delete()

AttributeError: 'NoneType' object has no attribute 'delete'

In [ ]:
Pessoa.objects(id='625f48b63a1f265e4af5816b')